<a href="https://colab.research.google.com/github/mosovam/nn-medical-view/blob/main/PyTorch_model_to_ONNX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sources

[Exporting a Model from PyTorch to ONNX and Running it using ONNX Runtime](https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/8c7f0be1e1c3803fcb4c41bcd9f4226b/super_resolution_with_onnxruntime.ipynb)

[ONNX Documentation - Example: AlexNet from PyTorch to ONNX](https://pytorch.org/docs/master/onnx.html#example-alexnet-from-pytorch-to-onnx)

# Imports and installations

In [ ]:
%pip install segmentation-models-pytorch

     |████████████████████████████████| 88 kB 3.4 MB/s 
     |████████████████████████████████| 58 kB 5.0 MB/s 
     |████████████████████████████████| 376 kB 36.0 MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12421 sha256=6a90fb558be4ce78c13103941d3320ccaf9e412327bc19b9e2159d4148e00470
  Stored in directory: /root/.cache/pip/wheels/90/6b/0c/f0ad36d00310e65390b0d4c9218ae6250ac579c92540c9097a
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=b9047dc32d063873ddd9719bb51490d4f3481f7bfed956006f8145c3a636b5b9
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built efficientnet-pytorch pretrainedmodels


In [ ]:
# Some standard imports

import io
import numpy as np
import segmentation_models_pytorch as smp

from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx

In [ ]:
# google drive connection

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Global constants and functions

In [ ]:
MODEL_PATH = '/content/drive/MyDrive/BP_AI_OSU'

# use the same parameters as for model training
ENCODER = 'se_resnext50_32x4d'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid'
DEVICE = 'cuda'

In [ ]:
# prepare the model with predefiined parameters used when training the model

def get_prepared_Unet(model_classes):

  prepared_model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(model_classes), 
    activation=ACTIVATION
  )

  return prepared_model;

In [ ]:
# export the model as onnx model

def export_model(nn_model, name):
  export_path = MODEL_PATH + "/onnx_models/" + name + ".onnx"
  input_spec = torch.randn(1, 3, 512, 512).cuda()

  torch.onnx.export(
    nn_model,                   # model being run
    input_spec,                 # model input (or a tuple for multiple inputs)
    export_path,                # where to save the model (can be a file or file-like object)
    opset_version=11,
    input_names = ['input'],    # the model's input names
    output_names = ['output'],  # the model's output names
  )

# Convert model for TUMOR segmentation

In [ ]:
# prepare the model with predefiined parameters used when training the model

CLASSES = ['tumor', 'unlabelled']
tumor_model = get_prepared_Unet(CLASSES)

In [ ]:
# import the specific model and eval()

EPOCH ='10'
FOLDER_PATH = MODEL_PATH + '/nn_epochs_tumor/tumor_model_epoch' + EPOCH
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

tumor_model.load_state_dict(torch.load(FOLDER_PATH))
tumor_model.to(device)
tumor_model.eval()

Unet(
  (encoder): SENetEncoder(
    (layer0): Sequential(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (layer1): Sequential(
      (0): SEResNeXtBottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [ ]:
if torch.cuda.is_available():
    print('Tumor model on cuda!')
    tumor_model.cuda()

Tumor model on cuda!


In [ ]:
export_model(tumor_model, 'brain_tumor_model')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
/usr/local/lib/python3.7/dist-packages/torch/onnx/symbolic_helper.py:385: UserWarning: You are trying to export the model with onnx:Upsample for ONNX opset version 9. This operator might cause results to not match the expected results by PyTorch.
ONNX's Upsample/Resize operator did not match Pytorch's Interpolation until opset 11. Attributes to determine how to transform the input were added in onnx:Resize in opset 11 to support Pytorch's behavior (like coordinate_transformation_mode and nearest_mode).
We recommend using opset 11 and above for models using this operator.
  "" + str(_export_onnx_opset_version) + ". "


# Convert model for EYE segmentation

In [ ]:
# prepare the model with predefiined parameters used when training the model

CLASSES = ['eye', 'unlabelled']

eyes_model = smp.Unet(
  encoder_name=ENCODER, 
  encoder_weights=ENCODER_WEIGHTS, 
  classes=len(CLASSES), 
  activation=ACTIVATION
)

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnext50_32x4d-a260b3a4.pth" to /root/.cache/torch/hub/checkpoints/se_resnext50_32x4d-a260b3a4.pth


  0%|          | 0.00/105M [00:00<?, ?B/s]

In [ ]:
# import the specific model and eval()

EPOCH = '19'
FOLDER_PATH = MODEL_PATH + '/nn_epochs_eyes_copy/eyes_model_epoch' + EPOCH

eyes_model.load_state_dict(torch.load(FOLDER_PATH))
eyes_model.eval()

Unet(
  (encoder): SENetEncoder(
    (layer0): Sequential(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (layer1): Sequential(
      (0): SEResNeXtBottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [ ]:
if torch.cuda.is_available():
    print('Eyes model on cuda!')
    eyes_model.cuda()

Eyes model on cuda!


In [ ]:
export_model(eyes_model, 'brain_eyes_model')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


# Convert model for BRAINSTEM segmentation

In [ ]:
# prepare the model with predefiined parameters used when training the model

CLASSES = ['brainstem', 'unlabelled']
brainstem_model = get_prepared_Unet(CLASSES)

In [ ]:
# import the specific model and eval()

EPOCH = '19'
FOLDER_PATH = MODEL_PATH + '/nn_epochs_brainstem/brainstem_model_epoch' + EPOCH

brainstem_model.load_state_dict(torch.load(FOLDER_PATH))
brainstem_model.eval()

Unet(
  (encoder): SENetEncoder(
    (layer0): Sequential(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (layer1): Sequential(
      (0): SEResNeXtBottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [ ]:
if torch.cuda.is_available():
    print('Brainstem model on cuda!')
    brainstem_model.cuda()

Brainstem model on cuda!


In [ ]:
export_model(brainstem_model, 'brain_brainstem_model')

/usr/local/lib/python3.7/dist-packages/torch/onnx/utils.py:286: UserWarning: `add_node_names' can be set to True only when 'operator_export_type' is `ONNX`. Since 'operator_export_type' is not set to 'ONNX', `add_node_names` argument will be ignored.
  "`{}` argument will be ignored.".format(arg_name, arg_name))
/usr/local/lib/python3.7/dist-packages/torch/onnx/utils.py:286: UserWarning: `do_constant_folding' can be set to True only when 'operator_export_type' is `ONNX`. Since 'operator_export_type' is not set to 'ONNX', `do_constant_folding` argument will be ignored.
  "`{}` argument will be ignored.".format(arg_name, arg_name))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
